In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from ipywidgets import IntProgress
from IPython.display import display

In [15]:
!gdown --id '1FXlXCQI2PlBDasMxLsNZvAarq9_DHomU' --output taxa-bar-plots-left-trim-level6.csv

Downloading...
From: https://drive.google.com/uc?id=1FXlXCQI2PlBDasMxLsNZvAarq9_DHomU
To: /content/taxa-bar-plots-left-trim-level6.csv
100% 103k/103k [00:00<00:00, 902kB/s]


In [16]:
train = pd.read_csv("taxa-bar-plots-left-trim-level6.csv",index_col='number')
train.head()

,index,Bacteria;Abditibacteriota;Abditibacteria;Abditibacteriales;Abditibacteriaceae;Abditibacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Corynebacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Lawsonella,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Gordonia,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Sporichthyaceae;uncultured,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Ornithinicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;__,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Micrococcaceae;Enteractinococcus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Micrococcaceae;Kocuria,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Micrococcaceae;Micrococcus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Micrococcaceae;Rothia,Bacteria;Actinobacteriota;Actinobacteria;Propionibacteriales;Nocardioidaceae;Nocardioides,Bacteria;Actinobacteriota;Actinobacteria;Propionibacteriales;Propionibacteriaceae;Cutibacterium,Bacteria;Actinobacteriota;Actinobacteria;Propionibacteriales;Propionibacteriaceae;Pseudopropionibacterium,Bacteria;Actinobacteriota;Actinobacteria;Streptomycetales;Streptomycetaceae;Streptomyces,Bacteria;Actinobacteriota;Rubrobacteria;Rubrobacterales;Rubrobacteriaceae;Rubrobacter,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Bacteroidaceae;Bacteroides,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Paludibacteraceae;F0058,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Porphyromonadaceae;Porphyromonas,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Prevotellaceae;Alloprevotella,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Prevotellaceae;Prevotella,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Tannerellaceae;Tannerella,Bacteria;Bacteroidota;Bacteroidia;Chitinophagales;Chitinophagaceae;Flavihumibacter,Bacteria;Bacteroidota;Bacteroidia;Chitinophagales;Chitinophagaceae;Sediminibacterium,Bacteria;Bacteroidota;Bacteroidia;Cytophagales;Hymenobacteraceae;Hymenobacter,Bacteria;Bacteroidota;Bacteroidia;Cytophagales;Hymenobacteraceae;Pontibacter,Bacteria;Bacteroidota;Bacteroidia;Cytophagales;Spirosomaceae;Dyadobacter,Bacteria;Bacteroidota;Bacteroidia;Cytophagales;Spirosomaceae;Pseudarcicella,Bacteria;Bacteroidota;Bacteroidia;Cytophagales;Spirosomaceae;Spirosoma,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Flavobacteriaceae;Capnocytophaga,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Flavobacteriaceae;Flavobacterium,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Weeksellaceae;Bergeyella,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Weeksellaceae;Chryseobacterium,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Weeksellaceae;Cloacibacterium,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Weeksellaceae;Empedobacter,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Weeksellaceae;__,Bacteria;Bacteroidota;Bacteroidia;Sphingobacteriales;Sphingobacteriaceae;Mucilaginibacter,Bacteria;Bacteroidota;Bacteroidia;Sphingobacteriales;Sphingobacteriaceae;Pedobacter,Bacteria;Bdellovibrionota;Bdellovibrionia;Bdellovibrionales;Bdellovibrionaceae;Bdellovibrio,...,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Enterobacteriaceae;__,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Erwiniaceae;Erwinia,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Erwiniaceae;__,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Yersiniaceae;Serratia,Bacteria;Proteobacteria;Gammaproteobacteria;Legionellales;Legionellaceae;Legionella,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Actinobacillus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Aggregatibacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Haemophilus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Pasteurella,Bacteria;Proteo

In [33]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168 entries, 1 to 168
Columns: 224 entries, index to smoker
dtypes: float64(2), int64(218), object(4)
memory usage: 300.3+ KB


## merge read最短的9個run，在training時把他們去掉(去掉短於210的)
SRR10718013 length = 42 index=72  
SRR10718047 length = 47 index=106    
SRR10718024 length = 79 index=83  
SRR10717991 length = 111 index=50  
SRR10718078 length = 123 index=137   
SRR10718037 length = 147 index=96  
SRR10718067 length = 168 index=126  
SRR10717982 length = 191 index=41  
SRR10717997 length = 204 index=56  

In [34]:
# shortrun = ["SRR10718013", "SRR10718047", "SRR10718024", "SRR10717991", "SRR10718078", "SRR10718037", "SRR10718067", "SRR10717982", "SRR10717997"]
# shortrun = ["41", "50", "56", "72", "83", "96", "106", "126", "137"]
shortrun = train[train['merged'] < 210].index
train = train.drop(shortrun,axis=0)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 1 to 168
Columns: 224 entries, index to smoker
dtypes: float64(2), int64(218), object(4)
memory usage: 279.5+ KB


In [35]:
CAT_COL = ["index", "sex", "smoker", "Clinical"]

NUM_COL=[]
for i in range(len(train.columns)):
  NUM_COL.append(train.columns[i])
NUM_COL.remove('index')
NUM_COL.remove('Clinical')
NUM_COL.remove('sex')
NUM_COL.remove('smoker')

cat_col = []
num_col = []
for col in train:
  if col in CAT_COL:
    cat_col.append(col)
  elif col in NUM_COL:
    num_col.append(col)
for col in cat_col:
  train[col] = train[col].astype(str)

df_cat = train.loc[:,cat_col] # take all the categorical columns
df_cat = pd.get_dummies(df_cat) # one hot encoding

df_num = train.loc[:,num_col] # take all the numerical columns
df_final = pd.concat([df_cat, df_num], axis=1) # concat categorical/numerical data

In [36]:
df_final.head()

,index_SRR10717942,index_SRR10717943,index_SRR10717944,index_SRR10717945,index_SRR10717946,index_SRR10717947,index_SRR10717948,index_SRR10717949,index_SRR10717950,index_SRR10717951,index_SRR10717952,index_SRR10717953,index_SRR10717954,index_SRR10717955,index_SRR10717956,index_SRR10717957,index_SRR10717958,index_SRR10717959,index_SRR10717960,index_SRR10717961,index_SRR10717962,index_SRR10717963,index_SRR10717964,index_SRR10717965,index_SRR10717966,index_SRR10717967,index_SRR10717968,index_SRR10717969,index_SRR10717970,index_SRR10717971,index_SRR10717972,index_SRR10717973,index_SRR10717974,index_SRR10717975,index_SRR10717976,index_SRR10717977,index_SRR10717978,index_SRR10717979,index_SRR10717980,index_SRR10717981,...,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Enterobacteriaceae;Escherichia-Shigella,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Enterobacteriaceae;Klebsiella,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Enterobacteriaceae;Kluyvera,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Enterobacteriaceae;__,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Erwiniaceae;Erwinia,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Erwiniaceae;__,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Yersiniaceae;Serratia,Bacteria;Proteobacteria;Gammaproteobacteria;Legionellales;Legionellaceae;Legionella,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Actinobacillus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Aggregatibacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Haemophilus,Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Pasteurella,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Acinetobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Alkanindiges,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Enhydrobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Moraxella,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;Psychrobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;__,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Moraxellaceae;uncultured,Bacteria;Proteobacteria;Gammaproteobacteria;Pseudomonadales;Pseudomonadaceae;Pseudomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Vibrionales;Vibrionaceae;Photobacterium,Bacteria;Proteobacteria;Gammaproteobacteria;Vibrionales;Vibrionaceae;Vibrio,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Xanthomonadaceae;Luteimonas,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Xanthomonadaceae;Stenotrophomonas,Bacteria;Proteobacteria;Gammaproteobacteria;Xanthomonadales;Xanthomonadaceae;Xanthomonas,Bacteria;Proteobacteria;Gammaproteobacteria;__;__;__,Bacteria;Proteobacteria;__;__;__;__,Bacteria;SAR324_clade(Marine_group_B);uncultured_bacterium;;__;__,Bacteria;Spirochaetota;Spirochaetia;Spirochaetales;Spirochaetaceae;Treponema,Bacteria;Verrucomicrobiota;Chlamydiae;Chlamydiales;__;__,Bacteria;Verrucomicrobiota;Omnitrophia;Omnitrophales;Omnitrophaceae;Candidatus_Omnitrophus,Bacteria;__;__;__;__;__,input,filtered,denoised,merged,non-chimeric,AGE,BMI,HDRS
number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,42,0,0,0,134,0,0,0,0,0,0,0,0,98,0,0,0,0,0,0,0,0,47478,1145,1140,1007,1007,33.9603,19.9,26
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,112,0,0,0,0,0,0,0,0,345,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50393,1440,1412,941,941,50.0479,23.5,25
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,52006,710,704,552,552,59.6

In [37]:
not_select = ["index", "Clinical", "input", "filtered", "denoised", "non-chimeric"]
train_select = train.drop(not_select,axis=1)
train_select.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 1 to 168
Columns: 218 entries, Bacteria;Abditibacteriota;Abditibacteria;Abditibacteriales;Abditibacteriaceae;Abditibacterium to smoker
dtypes: float64(2), int64(214), object(2)
memory usage: 272.0+ KB


In [38]:
cat_col = []
num_col = []
for col in train_select:
  if col in CAT_COL:
    cat_col.append(col)
  elif col in NUM_COL:
    num_col.append(col)

for col in cat_col:
  if train_select[col].dtype != "O":
    # print(col)
    train_select[col] = train_select[col].astype(str)

df_cat_select = train_select.loc[:,cat_col] # take all the categorical columns
df_cat_select = pd.get_dummies(df_cat_select) # one hot encoding

df_num_select = train_select.loc[:,num_col] # take all the numerical columns
df_final_select = pd.concat([df_cat_select, df_num_select], axis=1) # concat categorical/numerical data

In [39]:
df_final_select.head()

,sex_female,sex_male,smoker_No,smoker_Yes,Bacteria;Abditibacteriota;Abditibacteria;Abditibacteriales;Abditibacteriaceae;Abditibacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Corynebacterium,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Corynebacteriaceae;Lawsonella,Bacteria;Actinobacteriota;Actinobacteria;Corynebacteriales;Nocardiaceae;Gordonia,Bacteria;Actinobacteriota;Actinobacteria;Frankiales;Sporichthyaceae;uncultured,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;Ornithinicoccus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Intrasporangiaceae;__,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Micrococcaceae;Enteractinococcus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Micrococcaceae;Kocuria,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Micrococcaceae;Micrococcus,Bacteria;Actinobacteriota;Actinobacteria;Micrococcales;Micrococcaceae;Rothia,Bacteria;Actinobacteriota;Actinobacteria;Propionibacteriales;Nocardioidaceae;Nocardioides,Bacteria;Actinobacteriota;Actinobacteria;Propionibacteriales;Propionibacteriaceae;Cutibacterium,Bacteria;Actinobacteriota;Actinobacteria;Propionibacteriales;Propionibacteriaceae;Pseudopropionibacterium,Bacteria;Actinobacteriota;Actinobacteria;Streptomycetales;Streptomycetaceae;Streptomyces,Bacteria;Actinobacteriota;Rubrobacteria;Rubrobacterales;Rubrobacteriaceae;Rubrobacter,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Bacteroidaceae;Bacteroides,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Paludibacteraceae;F0058,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Porphyromonadaceae;Porphyromonas,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Prevotellaceae;Alloprevotella,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Prevotellaceae;Prevotella,Bacteria;Bacteroidota;Bacteroidia;Bacteroidales;Tannerellaceae;Tannerella,Bacteria;Bacteroidota;Bacteroidia;Chitinophagales;Chitinophagaceae;Flavihumibacter,Bacteria;Bacteroidota;Bacteroidia;Chitinophagales;Chitinophagaceae;Sediminibacterium,Bacteria;Bacteroidota;Bacteroidia;Cytophagales;Hymenobacteraceae;Hymenobacter,Bacteria;Bacteroidota;Bacteroidia;Cytophagales;Hymenobacteraceae;Pontibacter,Bacteria;Bacteroidota;Bacteroidia;Cytophagales;Spirosomaceae;Dyadobacter,Bacteria;Bacteroidota;Bacteroidia;Cytophagales;Spirosomaceae;Pseudarcicella,Bacteria;Bacteroidota;Bacteroidia;Cytophagales;Spirosomaceae;Spirosoma,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Flavobacteriaceae;Capnocytophaga,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Flavobacteriaceae;Flavobacterium,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Weeksellaceae;Bergeyella,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Weeksellaceae;Chryseobacterium,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Weeksellaceae;Cloacibacterium,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Weeksellaceae;Empedobacter,Bacteria;Bacteroidota;Bacteroidia;Flavobacteriales;Weeksellaceae;__,...,Bacteria;Proteobacteria;Gammaproteobacteria;Cardiobacteriales;Cardiobacteriaceae;Cardiobacterium,Bacteria;Proteobacteria;Gammaproteobacteria;Cellvibrionales;Cellvibrionaceae;Cellvibrio,Bacteria;Proteobacteria;Gammaproteobacteria;Diplorickettsiales;Diplorickettsiaceae;uncultured,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Enterobacteriaceae;Enterobacter,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Enterobacteriaceae;Escherichia-Shigella,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Enterobacteriaceae;Klebsiella,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Enterobacteriaceae;Kluyvera,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Enterobacteriaceae;__,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Erwiniaceae;Erwinia,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Erwiniaceae;__,Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Yersiniaceae;Serratia,Bacteria;Proteobacteria;Gammapro

#Random Forest Classifier

In [40]:
#Use RandomForestClassifier to predict Clinical
x = df_final_select
y = train["Clinical"]
# y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=0)

#RandomForest
rfc = RandomForestClassifier()
#rfc=RandomForestClassifier(n_estimators=100,n_jobs = -1,random_state =50, min_samples_leaf = 10)
rfc.fit(X_train,y_train)
y_predict = rfc.predict(X_train)
score_rfc = rfc.score(X_test,y_test)
print("Random Forest Accuracy = ",score_rfc*100," %")

Random Forest Accuracy =  56.25  %


In [41]:
from sklearn.model_selection import KFold
x = df_final_select
y = train["Clinical"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)
    y_predict = rfc.predict(X_train)
    accuracy = rfc.score(X_test,y_test)
    print("Random Forest Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_rfc = rfc
print("Best Accuracy = ",best_accuracy*100,"%")

Random Forest Accuracy =  56.25 %
Random Forest Accuracy =  59.375 %
Random Forest Accuracy =  93.75 %
Random Forest Accuracy =  53.125 %
Random Forest Accuracy =  48.38709677419355 %
Best Accuracy =  93.75 %


In [42]:
#random forest final accuracy
x_train_new = x
y_train_new = train["Clinical"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred = best_rfc.predict(x_train_new)
count = 0

for i in range(y_pred.shape[0]):
    if y_pred[i] != y_train_new[i]:
        count += 1
    # print(y_pred[i],y_train_new[i])
rfc_accuracy = 1-count/y_pred.shape[0]
print("Random Forest Accuracy = ",rfc_accuracy*100,"%")

Random Forest Accuracy =  98.74213836477988 %


#SVM

In [43]:
from sklearn import svm
#Use SVM to predict Clinical
x = df_final_select
y = train["Clinical"]
# y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=0)

clf = svm.SVC()
clf.fit(X_train,y_train)
y_predict = clf.predict(X_train)
score_clf = clf.score(X_test,y_test)
print("SVM Accuracy = ",score_clf*100," %")

SVM Accuracy =  25.0  %


In [44]:
from sklearn.model_selection import KFold
x = df_final_select
y = train["Clinical"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    clf = svm.SVC()
    clf.fit(X_train,y_train)
    y_predict = clf.predict(X_train)
    accuracy = clf.score(X_test,y_test)
    print("SVM Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_clf = clf
print("Best Accuracy = ",best_accuracy*100,"%")

SVM Accuracy =  12.5 %
SVM Accuracy =  28.125 %
SVM Accuracy =  18.75 %
SVM Accuracy =  21.875 %
SVM Accuracy =  22.58064516129032 %
Best Accuracy =  28.125 %


In [45]:
#SVM final accuracy
x_train_new = x
y_train_new = train["Clinical"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred = best_clf.predict(x_train_new)
count = 0

for i in range(y_pred.shape[0]):
    # print(y_pred[i],y_train_new[i])
    if y_pred[i] != y_train_new[i]:
        count += 1
clf_accuracy = 1-count/y_pred.shape[0]
print("SVM Accuracy = ",clf_accuracy*100,"%")

SVM Accuracy =  46.540880503144656 %


#Neural network MLPClassifier

In [46]:
from sklearn.neural_network import MLPClassifier
#Use Neural Network MLPClassifier to predict Clinical
x = df_final_select
y = train["Clinical"]
# y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=0)

nnclf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50, 30), random_state=1, max_iter=2000)
nnclf.fit(X_train,y_train)
y_predict = nnclf.predict(X_train)
score_nnclf = nnclf.score(X_test,y_test)
print("Neural Network Accuracy = ",score_nnclf*100," %")

Neural Network Accuracy =  18.75  %


In [47]:
from sklearn.model_selection import KFold
x = df_final_select
y = train["Clinical"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    nnclf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50, 30), random_state=1, max_iter=2000)
    nnclf.fit(X_train,y_train)
    y_predict = nnclf.predict(X_train)
    accuracy = nnclf.score(X_test,y_test)
    print("NN Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_nnclf = nnclf
print("NN Accuracy = ",best_accuracy*100,"%")

NN Accuracy =  37.5 %
NN Accuracy =  28.125 %
NN Accuracy =  18.75 %
NN Accuracy =  46.875 %
NN Accuracy =  41.935483870967744 %
NN Accuracy =  46.875 %


In [48]:
#NN final accuracy
x_train_new = x
y_train_new = train["Clinical"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred = best_nnclf.predict(x_train_new)
count = 0

for i in range(y_pred.shape[0]):
    # print(y_pred[i],y_train_new[i])
    if y_pred[i] != y_train_new[i]:
        count += 1
nnclf_accuracy = 1-count/y_pred.shape[0]
print("NN Accuracy = ",nnclf_accuracy*100,"%")

NN Accuracy =  89.30817610062893 %


#Logistic Regression

In [49]:
from sklearn.linear_model import LogisticRegression
#Use Logistic Regression to predict Clinical
x = df_final_select
y = train["Clinical"]
# y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=0)

logclf = LogisticRegression(random_state=0).fit(X_train,y_train)
logclf.predict(X_train)
logclf.predict_proba(X_train)
score_logclf = logclf.score(X_test,y_test)
print("Logistic Regression Accuracy = ",score_logclf*100," %")

Logistic Regression Accuracy =  31.25  %


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [50]:
from sklearn.model_selection import KFold
x = df_final_select
y = train["Clinical"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x.iloc[train_index], x.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    logclf = LogisticRegression(random_state=0).fit(X_train,y_train)
    y_predict = logclf.predict(X_train)
    accuracy = logclf.score(X_test,y_test)
    print("Logistic Regression Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_logclf = logclf
print("Logistic Regression Accuracy = ",best_accuracy*100,"%")

Logistic Regression Accuracy =  31.25 %
Logistic Regression Accuracy =  18.75 %
Logistic Regression Accuracy =  40.625 %
Logistic Regression Accuracy =  37.5 %
Logistic Regression Accuracy =  45.16129032258064 %
Logistic Regression Accuracy =  45.16129032258064 %


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [51]:
#Logistic final accuracy
x_train_new = x
y_train_new = train["Clinical"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred = best_logclf.predict(x_train_new)
count = 0

for i in range(y_pred.shape[0]):
    # print(y_pred[i],y_train_new[i])
    if y_pred[i] != y_train_new[i]:
        count += 1
logclf_accuracy = 1-count/y_pred.shape[0]
print("Logistic Regression Accuracy = ",logclf_accuracy*100,"%")

Logistic Regression Accuracy =  89.30817610062893 %
